# Lyra

## Riemann convention

`TensorSpace` and `SpaceTime` accept a `riemann_convention` parameter to choose the overall
sign of the Riemann/Ricci/scalar curvature tensors. Default is `"mtw"`/`"wald"`,
while `"landau-lifshitz"`/`"weinberg"` flips the sign.

```python
st = pl.SpaceTime(coords=(x, y), metric=sp.diag(1, 1), riemann_convention="landau-lifshitz")
```


In [5]:
!pip install -U lyra-geometry -qqq

## Quick tutorial: exploring the `lyra_geometry` API

This tutorial walks through the main features of the library, from creating a space with a metric to index manipulation, contractions, covariant derivatives, and curvature tensors.


In [1]:
import sympy as sp
import lyra_geometry as pl

# Tip: if you want to use the name `d` as an index label,
# keep an alias before overwriting pl.d.
dn = pl.d


### 1) Creating a space with a metric

`SpaceTime` (or `TensorSpace`) defines coordinates, a metric, and derived structures (determinant, connection, curvature).

In [2]:
x, y = sp.symbols("x y", real=True)
coords = (x, y)

# Simple 2D example: metric diag(1, x^2)
metric = sp.diag(x+2*y, x**2*y)

st = pl.SpaceTime(coords=coords, metric=metric)

st.g  # metric (rank-2 covariant tensor)


In [3]:
st.metric_inv  # inverse metric matrix
st.detg       # determinant of the metric


x**2*y*(x + 2*y)

### 2) Indices, raising/lowering, and components

Use `+a/-a` (or `u(a)/d(a)`) to indicate variance. The `index` method creates labels for automatic contraction.

In [4]:
a, b, c = st.index("a b c")

# Components with indices
st.g[-a, -b]          # g_ab
st.g[+a, +b]          # g^ab (uses the inverse metric)

# Access a specific component
st.g[-a, -b](0, 0)


x + 2*y

### 3) Generic tensors and contraction

Create symbolic tensors and contract repeated indices automatically.

In [5]:
v = st.tensor.generic("v", (pl.U,))
w = st.tensor.generic("w", (pl.D,))

scalar = v[+a] * w[-a]  # automatic contraction
scalar


In [6]:
# Explicit contraction via TensorSpace.contract
st.contract(v[+a], w[-a])


In [7]:
# Simple parser for index strings
st.eval_contract("v^a w_a")


### 4) Lyra covariant derivative

The covariant derivative respects the space connection. The result gains one extra covariant index.

In [8]:
dv = st.nabla(v)

# dv has signature (D, U)
dv.signature


(<lyra_geometry.core.Down at 0x7c252bddb620>,
 <lyra_geometry.core.Up at 0x7c252bddae40>)

In [9]:
# Access components of dv
# (first index: derivative, second: original v index)
dv[-b, +a](0, 0)


((x + 2*y)*Derivative(v0(x, y), x) + v0(x, y)/2 + v1(x, y))/(x + 2*y)

In [ ]:
# Gradient, divergence, and Laplacian examples
f = st.scalar(x**2 + y**2)
grad_f = st.gradient(f)
div_v = st.divergence(v)
lap_f = st.laplacian(f)
grad_f, div_v, lap_f


### 5) Connection, curvature, and derived tensors

When a metric is provided, the Lyra connection is computed automatically.

In [10]:
st.gamma      # Lyra connection (Gamma^a_{bc})


In [11]:
st.riemann    # Riemann tensor
st.ricci      # Ricci tensor
st.einstein   # Einstein tensor
st.scalar_curvature


### 6) Scale (phi), torsion, and non-metricity

You can set a scale function and provide torsion/non-metricity explicitly.

In [12]:
phi = sp.Function("phi")(x)
st.set_scale(phi)

# Example: torsion and non-metricity set to zero
st.set_torsion(st.zeros((pl.D, pl.D, pl.D)))
st.set_nonmetricity(st.zeros((pl.U, pl.D, pl.D)))

st.update()


### 7) Custom connection strategies

If you already have Gamma components, you can fix the connection manually.

In [13]:
# Zero connection (Gamma = 0) in 2D
Gamma0 = sp.ImmutableDenseNDimArray([0] * (2**3), (2, 2, 2))

st2 = pl.SpaceTime(
    coords=(x, y),
    metric=sp.diag(1, 1),
    connection_strategy=pl.FixedConnectionStrategy(Gamma0),
)

st2.gamma


### 8) Simplification with `fmt`

Use `fmt()` to expand and simplify expressions.

In [14]:
# fmt works with Tensor, IndexedTensor, or SymPy expressions
st.ricci.fmt()            # simplify the full tensor
st.einstein.fmt()         # idem
st.scalar_curvature.fmt() # scalar


### 9) Final tips

- Use `pl.greek("alpha")` if you want Greek index labels.
- For rank-0 tensors, `Tensor.expr` returns the underlying SymPy expression.
- `Tensor.as_signature` lets you raise/lower indices manually when needed.


In [4]:
import sympy as sp
import lyra_geometry as pl

# Schwarzschild Space-Time - General Relativity

In [15]:
import sympy as sp
import lyra_geometry as pl

# -- Metrics --------
print("Defining metrics")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)

nu = sp.Function('nu')
lamb = sp.Function('lambda')

g = sp.diag(sp.exp(nu(r)), -sp.exp(lamb(r)), -r**2, -r**2*sp.sin(th)**2)  # g_{mu nu}

st = pl.SpaceTime(coords=x, metric=g)

st.update()

Defining metrics


In [16]:
a, b, m, n  = st.index("alpha beta mu nu")

st.g[-m, -n]

[[exp(nu(r)), 0, 0, 0], [0, -exp(lambda(r)), 0, 0], [0, 0, -r**2, 0], [0, 0, 0, -r**2*sin(theta)**2]]

In [17]:
st.g[+m, +n]

[[exp(-nu(r)), 0, 0, 0], [0, -exp(-lambda(r)), 0, 0], [0, 0, -1/r**2, 0], [0, 0, 0, -1/(r**2*sin(theta)**2)]]

In [18]:
st.g[+m, +n] * st.g[-n, -a]

In [19]:
st.gamma

In [20]:
st.riemann

In [21]:
st.ricci

In [22]:
st.scalar_curvature

In [23]:
st.einstein.fmt()

In [24]:
Ricc = st.ricci
R = st.scalar_curvature
g = st.g

(Ricc[-m,-n] - sp.Rational(1,2) * g[-m,-n]*R).fmt()

[[-exp(-lambda(r))*exp(nu(r))*Derivative(lambda(r), r)/r - exp(nu(r))/r**2 + exp(-lambda(r))*exp(nu(r))/r**2, 0, 0, 0], [0, -Derivative(nu(r), r)/r + exp(lambda(r))/r**2 - 1/r**2, 0, 0], [0, 0, r**2*exp(-lambda(r))*Derivative(lambda(r), r)*Derivative(nu(r), r)/4 - r**2*exp(-lambda(r))*Derivative(nu(r), r)**2/4 - r**2*exp(-lambda(r))*Derivative(nu(r), (r, 2))/2 + r*exp(-lambda(r))*Derivative(lambda(r), r)/2 - r*exp(-lambda(r))*Derivative(nu(r), r)/2, 0], [0, 0, 0, r**2*exp(-lambda(r))*sin(theta)**2*Derivative(lambda(r), r)*Derivative(nu(r), r)/4 - r**2*exp(-lambda(r))*sin(theta)**2*Derivative(nu(r), r)**2/4 - r**2*exp(-lambda(r))*sin(theta)**2*Derivative(nu(r), (r, 2))/2 + r*exp(-lambda(r))*sin(theta)**2*Derivative(lambda(r), r)/2 - r*exp(-lambda(r))*sin(theta)**2*Derivative(nu(r), r)/2]]

In [25]:
assert st.einstein.fmt() == (Ricc[-m,-n] - sp.Rational(1,2) * g[-m,-n]*R).fmt()

# FLRW Space-Time - General Relativity

In [2]:
import sympy as sp
import lyra_geometry as pl

# -- Metrics --------
print("Defining metrics")
t, x, y, z = sp.symbols('t x y z', real=True)
a = sp.Function('a')

g = sp.diag(1, -a(t)**2, -a(t)**2, -a(t)**2)

st = pl.SpaceTime(coords=(t, x, y, z), metric=g)

st.update()


Defining metrics


In [38]:
alpha, beta, mu, nu = st.index("alpha beta mu nu")

st.g[-mu, -nu]


[[1, 0, 0, 0], [0, -a(t)**2, 0, 0], [0, 0, -a(t)**2, 0], [0, 0, 0, -a(t)**2]]

In [39]:
st.g[+mu, +nu]


[[1, 0, 0, 0], [0, -1/a(t)**2, 0, 0], [0, 0, -1/a(t)**2, 0], [0, 0, 0, -1/a(t)**2]]

In [40]:
st.g[+mu, +nu] * st.g[-nu, -alpha]


In [41]:
st.gamma


In [42]:
st.riemann


In [43]:
st.ricci


In [44]:
st.scalar_curvature


In [45]:
st.einstein


In [46]:
st.detg


-a(t)**6

In [48]:
st.nabla(a(t))


# Spherically Symmetric LyST Solution - (https://arxiv.org/pdf/2104.06295)

In [71]:
import sympy as sp
import lyra_geometry as pl

coords = t, r, th, vph = sp.symbols('t r theta varphi', real=True)

alpha = sp.Function('alpha')

# -- SpaceTime ----------
st = pl.SpaceTime(
    coords=coords,
    metric=sp.diag(alpha(r), -1/alpha(r), -r**2, -r**2*sp.sin(th)**2)
)

# -- Index --------
al, b, g, d, e, m, n, l, s, h, k  = st.index("alpha beta gamma delta epsilon mu nu lambda sigma eta kappa")


# -- Escala de Lyra ----------
print("Defining scale")
phi = st.set_scale(sp.Function('phi')(r))
st.update()

Defining scale


In [79]:
from IPython.display import Markdown

display(Markdown("### Lyst Spherically Symmetrica Field Equations - (Eqs.(59) of 2104.06295)"))
display(sp.Eq((phi**2*st.einstein[-m,-n]/alpha(r)**2).fmt()(0,0),0))
display(sp.Eq((phi**2*alpha(r)*st.einstein[-m,-n]/alpha(r)).fmt()(1,1),0))
display(sp.Eq((phi**2*st.einstein[-m,-n]/(r**2*alpha(r))).fmt()(2,2),0))


### Lyst Spherically Symmetrica Field Equations - (Eqs.(59) of 2104.06295)

Eq(2*Derivative(phi(r), (r, 2))/phi(r) - Derivative(phi(r), r)**2/phi(r)**2 + Derivative(alpha(r), r)*Derivative(phi(r), r)/(alpha(r)*phi(r)) + 4*Derivative(phi(r), r)/(r*phi(r)) + Derivative(alpha(r), r)/(r*alpha(r)) + r**(-2) - 1/(r**2*alpha(r)), 0)

Eq(-3*Derivative(phi(r), r)**2/phi(r)**2 - Derivative(alpha(r), r)*Derivative(phi(r), r)/(alpha(r)*phi(r)) - 4*Derivative(phi(r), r)/(r*phi(r)) - Derivative(alpha(r), r)/(r*alpha(r)) - 1/r**2 + 1/(r**2*alpha(r)), 0)

Eq(-2*Derivative(phi(r), (r, 2))/phi(r) + Derivative(phi(r), r)**2/phi(r)**2 - Derivative(alpha(r), (r, 2))/(2*alpha(r)) - 2*Derivative(alpha(r), r)*Derivative(phi(r), r)/(alpha(r)*phi(r)) - 2*Derivative(phi(r), r)/(r*phi(r)) - Derivative(alpha(r), r)/(r*alpha(r)), 0)

### Alternatively, we can use the non-covariant expression - (Eq.(54) of 2104.06295)

In [76]:
riemann_gr = st.tensor(
    (
        st.christoffel2[+l, -al, -n].d(-m)
        - st.christoffel2[+l, -al, -m].d(-n)
        + st.christoffel2[+s, -al, -n] * st.christoffel2[+l, -s, -m]
        - st.christoffel2[+s, -al, -m] * st.christoffel2[+l, -s, -n]
    ),
    index=(+l, -al, -m, -n),
)

ricci_gr = riemann_gr[+l, -m, -n, -l]

r_gr = ricci_gr[+m, -m]

LHS_LyST = st.tensor(
    (
        (
            ricci_gr[-m, -n]
            - sp.Rational(1,2) * st.g[-m, -n] * r_gr
            + 2 * st.phi * st.nabla(st.nabla(st.phi))[-m,-n]
            - 2 * st.phi * st.g[-m,-n] * st.nabla(st.nabla(st.phi))[+l,-l]
            + 3 * st.g[-m,-n] * st.nabla(st.phi)[-l] * st.nabla(st.phi)[+l]
        )
    ).fmt(),
    index=(-m, -n),
)

display(sp.Eq(LHS_LyST[+m,+n].fmt()(0,0),0))

display(sp.Eq(LHS_LyST[-m,+n].fmt()(1,1),0))

display(sp.Eq((LHS_LyST[-m,-n]/(r**2*alpha(r))).fmt()(2,2),0))


Eq(2*Derivative(phi(r), (r, 2))/phi(r) - Derivative(phi(r), r)**2/phi(r)**2 + Derivative(alpha(r), r)*Derivative(phi(r), r)/(alpha(r)*phi(r)) + 4*Derivative(phi(r), r)/(r*phi(r)) + Derivative(alpha(r), r)/(r*alpha(r)) + r**(-2) - 1/(r**2*alpha(r)), 0)

Eq(3*alpha(r)*Derivative(phi(r), r)**2/phi(r)**2 + Derivative(alpha(r), r)*Derivative(phi(r), r)/phi(r) + 4*alpha(r)*Derivative(phi(r), r)/(r*phi(r)) + Derivative(alpha(r), r)/r + alpha(r)/r**2 - 1/r**2, 0)

Eq(-2*Derivative(phi(r), (r, 2))/phi(r) + Derivative(phi(r), r)**2/phi(r)**2 - Derivative(alpha(r), (r, 2))/(2*alpha(r)) - 2*Derivative(alpha(r), r)*Derivative(phi(r), r)/(alpha(r)*phi(r)) - 2*Derivative(phi(r), r)/(r*phi(r)) - Derivative(alpha(r), r)/(r*alpha(r)), 0)